### Part 2
### Dory's optimal route

In [3]:
# information about Dory's prefer
start = "44.127985, -92.148796"
end = "44.05431509462441, -92.04443552409354" # North Picnic area
# Minnesota Digital Elevation Model - 30 Meter Resolution
dem_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip"
# NLCD 2019 Land Cover, Minnesota
lc_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip"
# MN road
road_url ="https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/trans_roads_centerlines/shp_trans_roads_centerlines.zip"
# bridge
bridge_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/trans_bridges/shp_trans_bridges.zip"
# water
water_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/water_strahler_stream_order/shp_water_strahler_stream_order.zip'
# counties
county_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_counties_in_minnesota/shp_bdry_counties_in_minnesota.zip"

In [11]:
import requests
import zipfile
import os
import arcpy
import numpy as np
from arcpy.sa import *

#### download data

In [11]:
# download DEM and extract
dem = requests.get(dem_url)

with open (r"D:\arc1\lab2\part2\fgdb_elev_30m_digital_elevation_model.zip", "wb") as dem_df:
    dem_df.write(dem.content)

sub_dir = "fgdb_elev_30m_digital_elevation_model"
unzip_path = os.path.join(r"D:\arc1\lab2\part2", sub_dir)

os.makedirs(unzip_path, exist_ok = True)

with zipfile.ZipFile(r"D:\arc1\lab2\part2\fgdb_elev_30m_digital_elevation_model.zip", "r") as dem_zip:
    dem_zip.extractall(unzip_path)
    
print(f"Unzipped DEM data to {unzip_path}")

Unzipped PRISM data to D:\arc1\lab2\part2\fgdb_elev_30m_digital_elevation_model


In [13]:
# NLCD
lc = requests.get(lc_url)

with open (r"D:\arc1\lab2\part2\tif_biota_landcover_nlcd_mn_2019.zip", "wb") as lc_df:
    lc_df.write(lc.content)

sub_dir = "tif_biota_landcover_nlcd_mn_2019"
unzip_path = os.path.join(r"D:\arc1\lab2\part2", sub_dir)

os.makedirs(unzip_path, exist_ok = True)

with zipfile.ZipFile(r"D:\arc1\lab2\part2\tif_biota_landcover_nlcd_mn_2019.zip", "r") as lc_zip:
    lc_zip.extractall(unzip_path)
    
print(f"Unzipped NLCD data to {unzip_path}")

Unzipped PRISM data to D:\arc1\lab2\part2\tif_biota_landcover_nlcd_mn_2019


In [14]:
# road
r = requests.get(road_url)

with open (r"D:\arc1\lab2\part2\shp_trans_roads_centerlines.zip", "wb") as r_df:
    r_df.write(r.content)

sub_dir = "shp_trans_roads_centerlines"
unzip_path = os.path.join(r"D:\arc1\lab2\part2", sub_dir)

os.makedirs(unzip_path, exist_ok = True)

with zipfile.ZipFile(r"D:\arc1\lab2\part2\shp_trans_roads_centerlines.zip", "r") as r_zip:
    r_zip.extractall(unzip_path)
    
print(f"Unzipped road data to {unzip_path}")

Unzipped PRISM data to D:\arc1\lab2\part2\shp_trans_roads_centerlines


In [17]:
# bridge
b = requests.get(bridge_url)

with open (r"D:\arc1\lab2\part2\shp_trans_bridges.zip", "wb") as b_df:
    b_df.write(b.content)

sub_dir = "shp_trans_bridges"
unzip_path = os.path.join(r"D:\arc1\lab2\part2", sub_dir)

os.makedirs(unzip_path, exist_ok = True)

with zipfile.ZipFile(r"D:\arc1\lab2\part2\shp_trans_bridges.zip", "r") as b_zip:
    b_zip.extractall(unzip_path)
    
print(f"Unzipped bridge data to {unzip_path}")

Unzipped PRISM data to D:\arc1\lab2\part2\shp_trans_bridges


In [4]:
# water
water = requests.post(water_url)

with open (r"D:\arc1\lab2\part2\shp_water_strahler_stream_order.zip", "wb") as w_df:
    w_df.write(water.content)

sub_dir = "shp_water_strahler_stream_order"
unzip_path = os.path.join(r"D:\arc1\lab2\part2", sub_dir)

os.makedirs(unzip_path, exist_ok = True)

with zipfile.ZipFile(r"D:\arc1\lab2\part2\shp_water_strahler_stream_order.zip", "r") as w_zip:
    w_zip.extractall(unzip_path)
    
print(f"Unzipped water data to {unzip_path}")

Unzipped water data to D:\arc1\lab2\part2\shp_water_strahler_stream_order


In [6]:
# counties
c = requests.get(county_url)
with open (r"D:\arc1\lab2\part2\shp_bdry_counties_in_minnesota.zip", 'wb') as c_zip:
    c_zip.write(c.content)
    
sub_dir = "shp_bdry_counties_in_minnesota"
unzip_path = os.path.join(r"D:\arc1\lab2\part2", sub_dir)

os.makedirs(unzip_path, exist_ok = True)

with zipfile.ZipFile(r"D:\arc1\lab2\part2\shp_bdry_counties_in_minnesota.zip", "r") as c_ref:
    c_ref.extractall(unzip_path)

print(f"Unzipped county data to {unzip_path}")

Unzipped county data to D:\arc1\lab2\part2\shp_bdry_counties_in_minnesota


### prepare the study extent

In [8]:
# Create Study Extent (Feature Class to Feature Class)
arcpy.conversion.FeatureClassToFeatureClass(r"D:\arc1\lab2\part2\shp_bdry_counties_in_minnesota\mn_county_boundaries.shp", r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb", "StudyExtent", "CTY_NAME = 'Wabasha' Or CTY_NAME = 'Winona' Or CTY_NAME = 'Olmsted'", 'AREA "AREA" true true false 19 Double 0 0,First,#,mn_county_boundaries,AREA,-1,-1;PERIMETER "PERIMETER" true true false 19 Double 0 0,First,#,mn_county_boundaries,PERIMETER,-1,-1;CTYONLY_ "CTYONLY_" true true false 19 Double 0 0,First,#,mn_county_boundaries,CTYONLY_,-1,-1;CTYONLY_ID "CTYONLY_ID" true true false 19 Double 0 0,First,#,mn_county_boundaries,CTYONLY_ID,-1,-1;COUN "COUN" true true false 4 Short 0 4,First,#,mn_county_boundaries,COUN,-1,-1;CTY_NAME "CTY_NAME" true true false 20 Text 0 0,First,#,mn_county_boundaries,CTY_NAME,0,20;CTY_ABBR "CTY_ABBR" true true false 4 Text 0 0,First,#,mn_county_boundaries,CTY_ABBR,0,4;CTY_FIPS "CTY_FIPS" true true false 4 Short 0 4,First,#,mn_county_boundaries,CTY_FIPS,-1,-1;Shape_Leng "Shape_Leng" true true false 19 Double 0 0,First,#,mn_county_boundaries,Shape_Leng,-1,-1;Shape_Area "Shape_Area" true true false 19 Double 0 0,First,#,mn_county_boundaries,Shape_Area,-1,-1', '')

<Result 'D:\\arc1\\lab2\\part2\\Lab2_2\\Lab2_2.gdb\\StudyExtent'>

In [9]:
# Dissolve County Boundaries 
arcpy.management.Dissolve("StudyExtent", r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\StudyExtent_Dissolve", None, None, "MULTI_PART", "UNSPLIT_LINES")

<Result 'D:\\arc1\\lab2\\part2\\Lab2_2\\Lab2_2.gdb\\StudyExtent_Dissolve'>

### prepare water stream

In [11]:
# Clip Streams
arcpy.analysis.Clip("streams_with_strahler_stream_order", "StudyExtent_Dissolve", r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\waterstream_Clip", None)

<Result 'D:\\arc1\\lab2\\part2\\Lab2_2\\Lab2_2.gdb\\waterstream_Clip'>

In [14]:
# Feature to Raster 
arcpy.conversion.FeatureToRaster("waterstream_Clip", "SO_VALUE", r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\Reclass_waterstream")

<Result 'D:\\arc1\\lab2\\part2\\Lab2_2\\Lab2_2.gdb\\Reclass_waterstream'>

In [15]:
# Reclassify Streams
arcpy.ddd.Reclassify("Reclass_waterstream", "Value", "1 1;2 2;3 3;4 7;5 8;6 9;8 10", r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\Reclass_Streams", "DATA")

<Result 'D:\\arc1\\lab2\\part2\\Lab2_2\\Lab2_2.gdb\\Reclass_Streams'>

### prepar landcover

In [35]:
# Define the input raster dataset
farm_field = r"D:\arc1\lab2\part2\tif_biota_landcover_nlcd_mn_2019\NLCD_2019_Land_Cover.tif"

# Define the output clipped raster
output_clip = r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\LandCover_clip"

# Define the clipping feature
clip_feature = "StudyExtent_Dissolve"

# Perform the clip operation
arcpy.management.Clip(farm_field, clip_feature, output_clip)

<Result 'D:\\arc1\\lab2\\part2\\Lab2_2\\Lab2_2.gdb\\LandCover_clip'>

In [36]:
# Reclassify Landcover
#farm_field_reclass = arcpy.sa.Reclassify("LandCover_clip.tif", "Value", "1 81 1;82 2;83 255 1", "DATA")

# Reclassify NLCD
arcpy.ddd.Reclassify("LandCover_clip", "NLCD_Land", "'Open Water' 10;'Developed, Open Space' 2;'Developed, Low Intensity' 2;'Developed, Medium Intensity' 2;'Developed, High Intensity' 2;'Barren Land' 5;'Deciduous Forest' 7;'Evergreen Forest' 7;'Mixed Forest' 7;Shrub/Scrub 7;Herbaceous 7;Hay/Pasture 9;'Cultivated Crops' 9;'Woody Wetlands' 10;'Emergent Herbaceous Wetlands' 10", r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\Reclass_NLCD", "DATA")

<Result 'D:\\arc1\\lab2\\part2\\Lab2_2\\Lab2_2.gdb\\Reclass_NLCD'>

### prepare slope

In [38]:
# Define the input raster dataset
DEM = r"D:\arc1\lab2\part2\fgdb_elev_30m_digital_elevation_model\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m"

# Define the output clipped raster
output_clip = r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\DEM_clip"

# Define the clipping feature
clip_feature = "StudyExtent_Dissolve"

# Perform the clip operation
arcpy.management.Clip(DEM , clip_feature, output_clip)

<Result 'D:\\arc1\\lab2\\part2\\Lab2_2\\Lab2_2.gdb\\DEM_clip'>

In [39]:
# Calculate Slope
slope_name = r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\DEM_clip\slope"

slope = arcpy.sa.Slope("DEM_clip", "DEGREE", 1, "PLANAR", "METER")

slope.save(slope_name)

In [41]:
# Reclassify Slope
Reclass_slope = arcpy.sa.Reclassify("slope", "VALUE", "0 5 5;5 10 4;10 15 3;15 20 2;20 90 1", "DATA") 


### create start & end points

In [58]:
# Create start point
start = arcpy.Point(44.127985, -92.148796)

# Create a point geometry
start_point = arcpy.PointGeometry(start, arcpy.SpatialReference(4326))
# Save the point geometry to a feature class
output_fc = r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\Start_Point"

arcpy.CopyFeatures_management(start_point, output_fc)

<Result 'D:\\arc1\\lab2\\part2\\Lab2_2\\Lab2_2.gdb\\Start_Point'>

In [60]:
# Create end point
end = arcpy.Point(44.05431509462441, -92.04443552409354)

# Create a point geometry
end_point = arcpy.PointGeometry(end, arcpy.SpatialReference(4326))
# Save the point geometry to a feature class
output_fc = r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\End_Point"

arcpy.CopyFeatures_management(end_point, output_fc)

<Result 'D:\\arc1\\lab2\\part2\\Lab2_2\\Lab2_2.gdb\\End_Point'>

In [21]:
# create rasters from starting point and end point

#buffer to give pointts dimensions, I buffered with a 30 m radius to give at least 4 pixels to the points
arcpy.analysis.Buffer("Start_Point", r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\start_point_Buffer", "0.05 Decimal Degrees", "FULL", "ROUND", "NONE", None, "PLANAR")

In [ ]:
#Convert buffered points to rasters
arcpy.conversion.FeatureToRaster("start_point_Buffer", "Shape_Length", r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\Start_Point_Buffer_PolygonToRaster")

In [ ]:
#buffer end point
arcpy.analysis.Buffer("End_Point", r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\end_point_Buffer", "0.05 Decimal Degrees", "FULL", "ROUND", "NONE", None, "PLANAR")

#Convert buffered points to rasters
arcpy.conversion.FeatureToRaster("end_point_Buffer", "Shape_Length", r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\End_Point_Buffer_PolygonToRaster")

### cost surface model

In [22]:
# Set local variables for input rasters
inRaster1 = "Reclass_slope"
inRaster2 = "Reclass_NLCD"
inRaster3 = "Reclass_Streams"

# Remap values for each raster
remapslope = RemapValue([[1, 1], [2, 2], [3, 3], [4, 4], [5, 5], ["NODATA", "NODATA"]])
remapNLCD = RemapValue([[2, 2], [5, 5], [7, 7], [9, 9], [10, 10],["NODATA", "Restricted"]])
remapstream = RemapValue([[1, 1], [2, 2], [3, 3], [7, 7], [8, 8], [9, 9], [10, 10], ["NODATA", "NODATA"]])

# Create a Weighted Overlay table
myWOTable = WOTable([[inRaster1, 50, "VALUE", remapslope],
                     [inRaster2, 20, "VALUE", remapNLCD], 
                     [inRaster3, 30, "VALUE", remapstream]
                    ], [1, 10, 1])

# Execute WeightedOverlay
outWeightedOverlay = WeightedOverlay(myWOTable)

# Save the output
outWeightedOverlay.save(r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\weighted1")


In [19]:
# Cost Distance
Cost_Distance = arcpy.sa.CostDistance("Start_Point_Buffer_PolygonToRaster", "weighted1", None, r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\Direction_Raster", None, None, None, None, ''); Cost_Distance.save(r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\CostDis")

In [20]:
# Cost Path (Best Single) 
Cost_Path = arcpy.sa.CostPath(r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\End_RasterPoint", "CostDis", "Direction_Raster", "BEST_SINGLE", 'OBJECTID', 'FLOW_DIRECTION'); Cost_Path.save(r"D:\arc1\lab2\part2\Lab2_2\Lab2_2.gdb\CostPath1")